In [1]:
from csv import reader
from collections import defaultdict
from itertools import chain, combinations
from datetime import datetime
import random

In [2]:
# read the input file and create a set of items and a list of itemset
def readFile(fpath):
    itemsets = []
    itemset = set()

    with open(fpath, 'r') as file:
        csv_reader = reader(file)
        for line in csv_reader:
            if line == "": continue
            if line == " ": continue
            new_line = []
            for item in line:
                item = item.replace(" ", "") 
                new_line.append(item)
            record = set(new_line) 
            for item in record:
                itemset.add(frozenset([item]))
            itemsets.append(record)
    return itemset, itemsets

In [15]:
# get the frequent itemsets that satisfy the minimum support count threshold
def getSatisMinSup(itemSet, itemset_list, min_sup, global_freq_itemset_sup):
    freq_itemset = set()
    local_freq_itemset_sup = defaultdict(int)

    for item in itemSet:
        for itemSet in itemset_list:
            if item.issubset(itemSet):
                global_freq_itemset_sup[item] += 1
                local_freq_itemset_sup[item] += 1

    for item, sup_count in local_freq_itemset_sup.items():
        if sup_count >= min_sup:
            freq_itemset.add(item)        
    
    return freq_itemset

In [16]:
# joining step
def joining(itemset, length):
    candidate_itemset_list = []
    for i in itemset:
        for j in itemset:
            if len(i.union(j)) == length:
                candidate_itemset_list.append(i.union(j))
    return set(candidate_itemset_list)

In [17]:
# pruning step
def pruning(Ck_itemset, prev_Lk_itemset, length):
    pruned_itemset = Ck_itemset
    for itemset in prev_Lk_itemset:
        subsets = combinations(itemset, length)
        for subset in subsets:
            # if the subset is not in previous Lk_itesmest, then remove the itemset
            if frozenset(subset) not in prev_Lk_itemset:
                pruned_itemset.remove(itemset)
                break
    return pruned_itemset

In [18]:
def getSubsets(itemset):  
    s = list(itemset)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)))

In [19]:
def associationRule(global_freq_itemset, global_freq_itemset_sup, min_conf):
    rules = []
    for k, itemsets in global_freq_itemset.items():
        for itemset in itemsets:
            subsets = getSubsets(itemset) # get all the subsets of the itemset            
            for subset in subsets:
                confidence = float(global_freq_itemset_sup[itemset] / global_freq_itemset_sup[frozenset(subset)])
                if confidence >= min_conf:
                    rules.append([set(subset), set(itemset.difference(subset)), confidence]) # difference = itemset - subset
    return rules

In [23]:
def apriori(fpath, min_sup, min_conf, optimized = 0):
    C1_itemset, itemset_list = readFile(fpath)  
    
    if optimized == 2:
        itemset_list = random.sample(itemset_list, 100)
    
    global_freq_itemset = dict()
    global_freq_itemset_sup = defaultdict(int)
    L1_itemset = getSatisMinSup(C1_itemset, itemset_list, min_sup, global_freq_itemset_sup)
    Lk_itemset = L1_itemset
    if optimized == 1:
        itemset_list = reduceTransaction(itemset_list, Lk_itemset)
    k = 2

    # compute frequent itemsets
    while Lk_itemset:  
        global_freq_itemset[k-1] = Lk_itemset
        Ck_itemset = joining(Lk_itemset, k)    
        pruned_itemset = pruning(Ck_itemset, Lk_itemset, k-1)
        Lk_itemset = getSatisMinSup(pruned_itemset, itemset_list, min_sup, global_freq_itemset_sup)
        if optimized == 1:
            itemset_list = reduceTransaction(itemset_list, Lk_itemset)
        k = k + 1
        
    rules = associationRule(global_freq_itemset, global_freq_itemset_sup, min_conf)
    rules.sort(key = lambda x: x[2])

    print("global_freq_itemset")
    for k in global_freq_itemset:
        print(str(k) + '-itemset: ' + str(global_freq_itemset[k]))
#     print("rules")
#     for rule in rules:
#         print(str(rule[0]) + ' ⇒ ' + str(rule[1]) + ', conf = ' + str(round(rule[2], 3)))

In [24]:
def reduceTransaction(itemset_list, Lk_itemset):
    mark = False
    for itemSet in itemset_list:
        for itemset in Lk_itemset:
            if itemset.issubset(itemSet):
                mark = True
                break
        if mark == False:
            itemset_list.remove(itemSet)
    return itemset_list

In [25]:
# implement the Apriori algorithm and calculate the running time
starting_time = datetime.now()
apriori('./adult.csv', 0.6*30162, 0.7)
ending_time = datetime.now()
running_time = (ending_time - starting_time).total_seconds()
print("Running time is " + str(running_time) + "s")

global_freq_itemset
1-itemset: {frozenset({'White'}), frozenset({'Private'}), frozenset({'United-States'}), frozenset({'Male'}), frozenset({'<=50K'})}
2-itemset: {frozenset({'<=50K', 'White'}), frozenset({'United-States', '<=50K'}), frozenset({'United-States', 'Male'}), frozenset({'Private', 'White'}), frozenset({'United-States', 'Private'}), frozenset({'United-States', 'White'})}
Running time is 1.232037s


In [11]:
# optimize the Apriori algorithm by reducing the transaction data and calculate the running time
starting_time = datetime.now()
apriori('./adult.csv', 0.6*30162, 0.7, 1)
ending_time = datetime.now()
running_time = (ending_time - starting_time).total_seconds()
print("Running time is " + str(running_time) + "s")

0.0014
0.0015
0.4688333333333333
0.0022333333333333333
0.0014
0.029833333333333333
0.2226
0.0004333333333333333
0.012333333333333333
0.0033333333333333335
0.0019666666666666665
0.03506666666666667
0.0006333333333333333
0.25753333333333334
0.002266666666666667
0.10706666666666667
0.04356666666666666
0.13433333333333333
0.0030666666666666668
0.0011333333333333334
0.054233333333333335
0.10706666666666667
0.027333333333333334
0.0011
0.01806666666666667
0.7551333333333333
0.027966666666666667
0.0004
0.6793333333333333
0.0336
0.02033333333333333
0.0007
0.0009
0.11946666666666667
0.0304
0.0005666666666666667
0.3242
0.032966666666666665
0.006266666666666667
0.00046666666666666666
0.045
0.00046666666666666666
0.06553333333333333
0.005033333333333333
0.0028666666666666667
0.0006
0.0689
0.009533333333333333
0.0036333333333333335
0.031433333333333334
0.0358
0.5901333333333333
0.2715
0.0014
0.1346
0.0077
0.002266666666666667
0.2633333333333333
0.18893333333333334
0.0009
0.001
0.04686666666666667
0.

In [13]:
# optimize the Apriori algorithm by sampling the itemset list and calculate the running time
starting_time = datetime.now()
apriori('./adult.csv', 0.5*100, 0.7, 2)
ending_time = datetime.now()
running_time = (ending_time - starting_time).total_seconds()
print("Running time is " + str(running_time) + "s")

0.0014666666666666667
6.666666666666667e-05
0.0008
3.3333333333333335e-05
3.3333333333333335e-05
6.666666666666667e-05
0.0008666666666666666
0.0002666666666666667
0.00016666666666666666
0.00046666666666666666
3.3333333333333335e-05
0.0002
0.00016666666666666666
0.0001
0.002533333333333333
3.3333333333333335e-05
0.0023666666666666667
3.3333333333333335e-05
0.0001
0.0003333333333333333
0.00016666666666666666
0.0013333333333333333
0.00013333333333333334
3.3333333333333335e-05
0.0002666666666666667
0.00013333333333333334
0.00013333333333333334
0.00013333333333333334
0.0002666666666666667
0.0019
0.0009666666666666667
0.00036666666666666667
6.666666666666667e-05
0.0009666666666666667
0.0008333333333333334
3.3333333333333335e-05
0.00013333333333333334
0.0005333333333333334
0.0013
6.666666666666667e-05
6.666666666666667e-05
6.666666666666667e-05
0.0004
0.0005333333333333334
0.0001
0.002533333333333333
0.003
0.0002666666666666667
0.0015
0.0011
3.3333333333333335e-05
6.666666666666667e-05
0.0009